In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import datasets
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import random
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,accuracy_score
from sklearn.utils import resample

from data_preprocessing.utils import preprocess_categorical_to_binary, change_age_range
from models.utils import split_train_test_with_sklearn,prepare_for_modeling,downsampling
%matplotlib inline


In [2]:
df = pd.read_csv('../data/raw/KaggleV2-May-2016.csv')
df = preprocess_categorical_to_binary(df)
df = change_age_range(df)


/home/kasia/Python Projects/Medical-Appoitnment-No-Show/src/data_preprocessing/utils.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col][X[col]<0]=0
/home/kasia/Python Projects/Medical-Appoitnment-No-Show/src/data_preprocessing/utils.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col][X[col]>95]=95


Summary after Data Visualization
- Age less than 0 and more than 100 should be drop
- days between where appoitment was earlier than schedule should be drop
- Unnamed:0 can be drop
- Appoitment ID is unique and can be use in index

Data for split 
- we should avoid that we have the same person in train and test
- comparision for diffrent train and test


### First model without up/downsampling

In [ ]:
train,test = split_train_test_with_sklearn(df)
X_train,y_train = prepare_for_modeling(train)
X_test,y_test = prepare_for_modeling(test)


rfc_clf = RandomForestClassifier()

rfc_clf.fit(X_train.values,y_train)
p_train = rfc_clf.predict(X_train.values)
print(classification_report(y_train,p_train))
acc_score_train = accuracy_score(y_train,p_train) 
p_test = rfc_clf.predict(X_test)
print(classification_report(y_test,p_test))
acc_score_test = accuracy_score(y_test,p_test) 

a = pd.DataFrame(X_train.columns)
a['fi'] = rfc_clf.feature_importances_
a.sort_values(by='fi',ascending=False).iloc[:10]

## Dataset downsampling

In [ ]:
#Check if our dataset is balanced or not
ratio = df['No-show'].value_counts()/df['No-show'].value_counts()[1]
print('"No"/"Yes"-Ratio is equal to {}/{}'.format(round(ratio[0],2),ratio[1]))
# here we can see that we have much more 'No' than 'Yes'

In [ ]:
v0 = df[df["No-show"]==0].sample(1000)
v1 = df[df["No-show"]==1].sample(1000)
valid_df = pd.concat([v0,v1]) 

train_v = df[~df.index.isin(valid_df.index)]

In [3]:
# Display new class counts
#train_D['No-show'].value_counts().plot(kind='bar')
df = downsampling(df,22319) 
train,test = split_train_test_with_sklearn(df)
#train = downsampling(train,15020) 
#test = downsampling(test,7299) 
X_train,y_train = prepare_for_modeling(train)
X_test,y_test = prepare_for_modeling(test)
#with neptune.create_experiment(name='noshow-1', tags=['downsampled'], params={'abcd': 123}, upload_source_files='*.ipynb'):

#Model
rfc_clf = RandomForestClassifier()
rfc_clf.fit(X_train,y_train)
p_down = rfc_clf.predict(X_train)
clf_report_train = classification_report(y_train,p_down)

# How's our accuracy?
acc_score_train = accuracy_score(y_train,p_down) 

p_test_down = rfc_clf.predict(X_test)
clf_report_valid = classification_report(y_test,p_test_down)
acc_score_test = accuracy_score(y_test,p_test_down) 
    #neptune.send_metric("acc_train", acc_score)
    #neptune.send_text('clf_report_valid', clf_report_valid)
    
  

In [4]:
print("test {}, train {}".format(acc_score_test,acc_score_train) )

test 0.37349806530446, train 0.8403718193065168


In [7]:
X_train

,Gender,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,DaysBetween
42979,0,85,0,1,1,0,0,0,0
102322,0,50,0,1,1,0,0,1,14
61539,0,32,0,0,0,0,0,1,7
42739,0,15,0,0,0,0,0,0,0
85880,0,38,0,0,0,0,0,1,17
...,...,...,...,...,...,...,...,...,...
30492,1,60,0,0,0,1,0,1,6
53619,0,14,0,0,0,0,0,0,0
46099,1,18,0,0,0,0,0,0,0
4107,1,21,0,0,0,0,0,0,35


In [ ]:
a = pd.DataFrame(X_train.columns)
a['fi'] = rfc_clf.feature_importances_
a.sort_values(by='fi',ascending=False).iloc[:10]

In [ ]:
pd.DataFrame(y_train)[0].value_counts()

In [ ]:
print('-----------Unbalanced-----------')
print(classification_report(y_test,p_test))
print('-----------Downsampling-----------')
print(classification_report(y_test,p_test_down))


In [ ]:
df['No-show']

In [ ]:
import neptune

neptune.init('kasiacieslik/sandbox', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYzNhMDBkMTYtN2ZlNy00YTMyLWEzMmQtMWQ3OTk1ZTMwZTQ1In0=')





In [ ]:
with neptune.create_experiment():
    neptune.send_text("costam","costam")